In [4]:
import os
import importlib
from utility import Pi_util
from natsort import natsorted
import numpy as np
import pickle
import sys
import pandas as pd

path_to_lib: str = "/Users/zacharykelly/Documents/MATLAB/projects/combiExperiments/code/lightLogger/miniSpect"
sys.path.append(path_to_lib)
import MS_util
import importlib

In [2]:
experiment_path: str = "/Volumes/EXTERNAL1/actuallyFixedBuffer_0.1hz_0NDF"
use_mean_frame: bool = True

In [3]:
Pi_util.parse_chunks_pkl(experiment_path, use_mean_frame)

Loading chunk: 1/12
Loading chunk: 2/12
Loading chunk: 3/12
Loading chunk: 4/12
Loading chunk: 5/12
Loading chunk: 6/12
Loading chunk: 7/12
Loading chunk: 8/12
Loading chunk: 9/12
Loading chunk: 10/12
Loading chunk: 11/12
Loading chunk: 12/12
Parsing chunk: 1/12
Length of world vals: 4
Frame Buffer Shape: (2001,)
Captured Frames: 2001 | FPS: 200.08818353579954
Length of MS Vals: 3
Reading Buffer Shape Before: 1628 | Num readings: 11.0
Reading Buffer Shape After: 1480 | Num readings: 10.0
Reading num: 0/10.0
Reading num: 1/10.0
Reading num: 2/10.0
Reading num: 3/10.0
Reading num: 4/10.0
Reading num: 5/10.0
Reading num: 6/10.0
Reading num: 7/10.0
Reading num: 8/10.0
Reading num: 9/10.0
Captured Frames: 10 | FPS: 0.9578720102154648
Length of Pupil vals: 3
Frame Buffer Shape: (1189,)
Captured Frames: 1189 | FPS: 118.87958446390883
Parsing chunk: 2/12
Length of world vals: 4
Frame Buffer Shape: (1995,)
Captured Frames: 1995 | FPS: 199.49451120696227
Length of MS Vals: 3
Reading Buffer Shape

[{'W': {'frame_buffer': array([240.51833333, 240.53166667, 240.58083333, ..., 240.68      ,
          240.67      , 240.62333333]),
   'settings_buffer': array([[1.00e+00, 1.00e+02],
          [1.00e+00, 1.00e+02],
          [1.00e+00, 1.00e+02],
          ...,
          [2.00e+00, 4.84e+03],
          [2.00e+00, 4.84e+03],
          [2.00e+00, 4.84e+03]]),
   'num_frames_captured': 2001.0,
   'FPS': 200.08818353579954},
  'M': {'A':       0     1     2     3     4     5     6     7      8     9
   0  4846  2119  3300  3875  4249  3389  7025  2382  25459  1100
   1  4846  2118  3300  3875  4249  3389  7025  2382  25459  1102
   2  4845  2119  3299  3875  4249  3388  7026  2382  25460  1102
   3  4846  2118  3299  3875  4248  3387  7025  2382  25465  1102
   4  4846  2118  3299  3875  4248  3387  7026  2383  25461  1102
   5  4845  2120  3300  3875  4251  3388  7028  2384  25458  1102
   6  4845  2117  3299  3874  4247  3385  7024  2383  25459  1102
   7  4846  2125  3303  3879  4265  3